In [11]:
# Property Price Prediction Model
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os
from datetime import datetime

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

# Set display options
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")
print(f"Notebook run time: {datetime.now()}")

Libraries imported successfully!
Current working directory: c:\Users\zulfa\OneDrive\Desktop\Pendayagunaan-Aset-dan-Properti\Pendayagunaan-Aset-dan-Properti\notebooks
Notebook run time: 2025-07-03 16:22:03.066316


In [12]:
# Data Loading and Initial Exploration
print("Loading datasets...")

# Define data paths
land_data_path = '../data/raw/Dataset_Tanah_Surabaya.csv'
building_data_path = '../data/raw/Dataset_Bangunan_Surabaya.csv'

# Load datasets
try:
    df_land = pd.read_csv(land_data_path)
    print(f"Land dataset loaded successfully: {df_land.shape}")
    
    df_building = pd.read_csv(building_data_path)
    print(f"Building dataset loaded successfully: {df_building.shape}")
    
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    
# Display basic information about the datasets
print("\n" + "="*50)
print("LAND DATASET OVERVIEW")
print("="*50)
print(f"Shape: {df_land.shape}")
print("\nColumn names:")
print(df_land.columns.tolist())
print("\nFirst few rows:")
df_land.head()

Loading datasets...
Land dataset loaded successfully: (8000, 9)
Building dataset loaded successfully: (8980, 22)

LAND DATASET OVERVIEW
Shape: (8000, 9)

Column names:
['Kecamatan', 'Lokasi', 'Luas_m2', 'Sertifikat', 'Kondisi', 'Tipe_Iklan', 'NJOP_Rp_per_m2', 'Jumlah_Penduduk', 'Aksesibilitas']

First few rows:


,Kecamatan,Lokasi,Luas_m2,Sertifikat,Kondisi,Tipe_Iklan,NJOP_Rp_per_m2,Jumlah_Penduduk,Aksesibilitas
0,"Gubeng,""Jl. Tubagus Ismail, Gubeng, Surabaya"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Sawahan,""Jl. Siliwangi, Sawahan, Surabaya"",112...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Asemrowo,""Gg. Rajawali Barat, Asemrowo, Suraba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Karang Pilang,""Jalan Veteran, Karang Pilang, S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sawahan,""Gg. H.J Maemunah, Sawahan, Surabaya"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Building Dataset Overview
print("="*50)
print("BUILDING DATASET OVERVIEW")
print("="*50)
print(f"Shape: {df_building.shape}")
print("\nColumn names:")
print(df_building.columns.tolist())
print("\nFirst few rows:")
df_building.head()

BUILDING DATASET OVERVIEW
Shape: (8980, 22)

Column names:
['Kecamatan', 'Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Sertifikat', 'Daya Listrik', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Hadap', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Alamat', 'Tipe Iklan', 'Aksesibilitas', 'NJOP_Rp_per_m2', 'Tingkat_Keamanan']

First few rows:


,Kecamatan,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Hadap,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Alamat,Tipe Iklan,Aksesibilitas,NJOP_Rp_per_m2,Tingkat_Keamanan
0,Wonokromo,3,2,45,70,SHM - Sertifikat Hak Milik,1300.0,Tidak,Tidak,Unfurnished,2.0,Barat,Tidak,NaN,NaN,Tidak,Baru,"Jl. Wonokromo Raya No.103, Surabaya",Keduanya,Baik,1032000.0,Tinggi
1,Rungkut,3,2,48,2023,SHM - Sertifikat Hak Milik,1300.0,Ya,Ya,Unfurnished,2.0,Selatan,Ya,2 Mobil,PAM atau PDAM,Tidak,Baru,"Jl. Rungkut Raya No.180, Surabaya",Keduanya,Baik,1032000.0,Tinggi
2,Semampir,3,3,48,85,SHM - Sertifikat Hak Milik,1300.0,Ya,Ya,Unfurnished,3.0,Selatan,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,"Jl. Semampir Raya No.93, Surabaya",Keduanya,Baik,1274000.0,Rendah
3,Pakal,2,1,50,91,HGB - Hak Guna Bangunan,1300.0,Ya,Ya,Unfurnished,1.0,Barat,Ya,2 Mobil,Sumur Pompa,Ya,Baru,"Jl. Pakal Raya No.15, Surabaya",Disewa,Buruk,1032000.0,Rendah
4,Gayungan,2,2,51,50,SHM - Sertifikat Hak Milik,2200.0,Tidak,Tidak,Unfurnished,NaN,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Baru,"Jl. Gayungan Raya No.107, Surabaya",Disewa,Baik,1032000.0,Tinggi


In [14]:
# Data Quality Assessment
def assess_data_quality(df, dataset_name):
    """Assess the quality of the dataset"""
    print(f"\n{'='*50}")
    print(f"DATA QUALITY ASSESSMENT - {dataset_name}")
    print('='*50)
    
    print(f"Dataset shape: {df.shape}")
    
    print("\n1. Data Types:")
    print(df.dtypes)
    
    print("\n2. Missing Values:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing Percentage': missing_percent
    })
    print(missing_df[missing_df['Missing Count'] > 0])
    
    print("\n3. Basic Statistics:")
    print(df.describe())
    
    print("\n4. Unique Values per Column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
    
    return missing_df

# Assess both datasets
land_quality = assess_data_quality(df_land, "LAND DATASET")
building_quality = assess_data_quality(df_building, "BUILDING DATASET")


DATA QUALITY ASSESSMENT - LAND DATASET
Dataset shape: (8000, 9)

1. Data Types:
Kecamatan           object
Lokasi             float64
Luas_m2            float64
Sertifikat         float64
Kondisi            float64
Tipe_Iklan         float64
NJOP_Rp_per_m2     float64
Jumlah_Penduduk    float64
Aksesibilitas      float64
dtype: object

2. Missing Values:
                 Missing Count  Missing Percentage
Lokasi                    8000               100.0
Luas_m2                   8000               100.0
Sertifikat                8000               100.0
Kondisi                   8000               100.0
Tipe_Iklan                8000               100.0
NJOP_Rp_per_m2            8000               100.0
Jumlah_Penduduk           8000               100.0
Aksesibilitas             8000               100.0

3. Basic Statistics:
       Lokasi  Luas_m2  Sertifikat  Kondisi  Tipe_Iklan  NJOP_Rp_per_m2  \
count     0.0      0.0         0.0      0.0         0.0             0.0   
mean      N

In [15]:
# Exploratory Data Analysis and Visualization
def create_eda_plots(df, dataset_name, target_col=None):
    """Create exploratory data analysis plots"""
    
    print(f"\n{'='*50}")
    print(f"EXPLORATORY DATA ANALYSIS - {dataset_name}")
    print('='*50)
    
    # Identify numeric and categorical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    print(f"Numeric columns: {numeric_cols}")
    print(f"Categorical columns: {categorical_cols}")
    
    # Create visualizations
    if len(numeric_cols) > 0:
        # Distribution plots for numeric columns
        n_cols = min(3, len(numeric_cols))
        n_rows = (len(numeric_cols) + 2) // 3
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        else:
            axes = axes.flatten()
        
        for i, col in enumerate(numeric_cols):
            if i < len(axes):
                df[col].hist(bins=30, ax=axes[i], alpha=0.7)
                axes[i].set_title(f'Distribution of {col}')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Frequency')
        
        # Hide empty subplots
        for i in range(len(numeric_cols), len(axes)):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    # Correlation matrix for numeric columns
    if len(numeric_cols) > 1:
        plt.figure(figsize=(12, 8))
        correlation_matrix = df[numeric_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                   square=True, linewidths=0.5)
        plt.title(f'Correlation Matrix - {dataset_name}')
        plt.tight_layout()
        plt.show()
    
    # Target variable analysis if specified
    if target_col and target_col in df.columns:
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        df[target_col].hist(bins=50, alpha=0.7)
        plt.title(f'Distribution of {target_col}')
        plt.xlabel(target_col)
        plt.ylabel('Frequency')
        
        plt.subplot(1, 3, 2)
        df.boxplot(column=target_col, ax=plt.gca())
        plt.title(f'Box Plot of {target_col}')
        
        plt.subplot(1, 3, 3)
        df[target_col].plot(kind='density', alpha=0.7)
        plt.title(f'Density Plot of {target_col}')
        plt.xlabel(target_col)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n{target_col} Statistics:")
        print(f"Mean: {df[target_col].mean():,.2f}")
        print(f"Median: {df[target_col].median():,.2f}")
        print(f"Std: {df[target_col].std():,.2f}")
        print(f"Min: {df[target_col].min():,.2f}")
        print(f"Max: {df[target_col].max():,.2f}")

# Perform EDA on both datasets
# Note: We'll need to identify the target column (price) after examining the data
print("Starting EDA for both datasets...")

# For now, let's examine the column names to identify potential target variables
print("\nLand dataset columns:", df_land.columns.tolist())
print("Building dataset columns:", df_building.columns.tolist())

Starting EDA for both datasets...

Land dataset columns: ['Kecamatan', 'Lokasi', 'Luas_m2', 'Sertifikat', 'Kondisi', 'Tipe_Iklan', 'NJOP_Rp_per_m2', 'Jumlah_Penduduk', 'Aksesibilitas']
Building dataset columns: ['Kecamatan', 'Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Sertifikat', 'Daya Listrik', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Hadap', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Alamat', 'Tipe Iklan', 'Aksesibilitas', 'NJOP_Rp_per_m2', 'Tingkat_Keamanan']


In [16]:
# Data Preprocessing and Feature Engineering
def preprocess_data(df, dataset_name, target_column=None):
    """
    Comprehensive data preprocessing pipeline
    """
    print(f"\n{'='*50}")
    print(f"PREPROCESSING - {dataset_name}")
    print('='*50)
    
    df_processed = df.copy()
    
    print(f"Original shape: {df_processed.shape}")
    
    # 1. Handle missing values
    print("\n1. Handling missing values...")
    missing_before = df_processed.isnull().sum().sum()
    
    # For numeric columns, fill with median
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_processed[col].isnull().sum() > 0:
            median_val = df_processed[col].median()
            df_processed[col].fillna(median_val, inplace=True)
            print(f"  - Filled {col} missing values with median: {median_val}")
    
    # For categorical columns, fill with mode
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_processed[col].isnull().sum() > 0:
            mode_val = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else 'Unknown'
            df_processed[col].fillna(mode_val, inplace=True)
            print(f"  - Filled {col} missing values with mode: {mode_val}")
    
    missing_after = df_processed.isnull().sum().sum()
    print(f"Missing values: {missing_before} -> {missing_after}")
    
    # 2. Remove duplicates
    print("\n2. Removing duplicates...")
    duplicates_before = df_processed.duplicated().sum()
    df_processed.drop_duplicates(inplace=True)
    duplicates_after = df_processed.duplicated().sum()
    print(f"Duplicates: {duplicates_before} -> {duplicates_after}")
    
    # 3. Handle outliers (for numeric columns)
    print("\n3. Handling outliers...")
    for col in numeric_cols:
        if col != target_column:  # Don't remove outliers from target variable
            Q1 = df_processed[col].quantile(0.25)
            Q3 = df_processed[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers_before = ((df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)).sum()
            
            # Cap outliers instead of removing them
            df_processed[col] = np.where(df_processed[col] < lower_bound, lower_bound, df_processed[col])
            df_processed[col] = np.where(df_processed[col] > upper_bound, upper_bound, df_processed[col])
            
            if outliers_before > 0:
                print(f"  - Capped {outliers_before} outliers in {col}")
    
    # 4. Feature Engineering
    print("\n4. Feature engineering...")
    
    # Create area-related features if applicable
    if 'luas_tanah' in df_processed.columns and 'luas_bangunan' in df_processed.columns:
        df_processed['building_to_land_ratio'] = df_processed['luas_bangunan'] / (df_processed['luas_tanah'] + 1)
        print("  - Created building_to_land_ratio feature")
    
    # Create price per square meter if price and area columns exist
    price_cols = [col for col in df_processed.columns if 'harga' in col.lower() or 'price' in col.lower()]
    area_cols = [col for col in df_processed.columns if 'luas' in col.lower() or 'area' in col.lower()]
    
    if len(price_cols) > 0 and len(area_cols) > 0:
        price_col = price_cols[0]
        area_col = area_cols[0]
        df_processed[f'price_per_sqm'] = df_processed[price_col] / (df_processed[area_col] + 1)
        print(f"  - Created price_per_sqm feature using {price_col} and {area_col}")
    
    # 5. Encode categorical variables
    print("\n5. Encoding categorical variables...")
    le_dict = {}
    
    for col in categorical_cols:
        if df_processed[col].nunique() < 20:  # Only encode if reasonable number of categories
            le = LabelEncoder()
            df_processed[f'{col}_encoded'] = le.fit_transform(df_processed[col].astype(str))
            le_dict[col] = le
            print(f"  - Label encoded {col} ({df_processed[col].nunique()} categories)")
    
    print(f"\nFinal shape: {df_processed.shape}")
    
    return df_processed, le_dict

# Apply preprocessing to both datasets
print("Starting preprocessing...")

# We'll need to identify target columns first by examining the actual data
# Let's run the preprocessing with placeholder target columns for now
df_land_processed, land_encoders = preprocess_data(df_land, "LAND DATASET")
df_building_processed, building_encoders = preprocess_data(df_building, "BUILDING DATASET")

Starting preprocessing...

PREPROCESSING - LAND DATASET
Original shape: (8000, 9)

1. Handling missing values...
  - Filled Lokasi missing values with median: nan
  - Filled Luas_m2 missing values with median: nan
  - Filled Sertifikat missing values with median: nan
  - Filled Kondisi missing values with median: nan
  - Filled Tipe_Iklan missing values with median: nan
  - Filled NJOP_Rp_per_m2 missing values with median: nan
  - Filled Jumlah_Penduduk missing values with median: nan
  - Filled Aksesibilitas missing values with median: nan
Missing values: 64000 -> 64000

2. Removing duplicates...
Duplicates: 1 -> 0

3. Handling outliers...

4. Feature engineering...

5. Encoding categorical variables...

Final shape: (7999, 9)

PREPROCESSING - BUILDING DATASET
Original shape: (8980, 22)

1. Handling missing values...
  - Filled Daya Listrik missing values with median: 2200.0
  - Filled Jumlah Lantai missing values with median: 2.0
  - Filled Ruang Makan missing values with mode: Tidak

In [17]:
# Model Training and Evaluation Functions
def prepare_features_target(df, target_column, exclude_columns=None):
    """
    Prepare features and target for modeling
    """
    if exclude_columns is None:
        exclude_columns = []
    
    # Identify feature columns (exclude target and specified columns)
    feature_columns = [col for col in df.columns 
                      if col != target_column and col not in exclude_columns 
                      and df[col].dtype in ['int64', 'float64']]
    
    X = df[feature_columns]
    y = df[target_column]
    
    print(f"Features selected: {feature_columns}")
    print(f"Feature matrix shape: {X.shape}")
    print(f"Target shape: {y.shape}")
    
    return X, y, feature_columns

def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluate model performance
    """
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"\n{model_name} Performance:")
    print(f"  MAE:  {mae:,.2f}")
    print(f"  RMSE: {rmse:,.2f}")
    print(f"  R² Score: {r2:.4f}")
    
    return {
        'model_name': model_name,
        'mae': mae,
        'rmse': rmse,
        'r2_score': r2,
        'predictions': y_pred
    }

def train_models(X_train, X_test, y_train, y_test):
    """
    Train XGBoost, Random Forest, and CatBoost models
    """
    print("Training machine learning models...")
    print("="*50)
    
    results = {}
    
    # 1. Random Forest
    print("\n1. Training Random Forest...")
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    results['Random Forest'] = evaluate_model(rf_model, X_test, y_test, 'Random Forest')
    results['Random Forest']['model'] = rf_model
    
    # 2. XGBoost
    print("\n2. Training XGBoost...")
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train)
    results['XGBoost'] = evaluate_model(xgb_model, X_test, y_test, 'XGBoost')
    results['XGBoost']['model'] = xgb_model
    
    # 3. CatBoost
    print("\n3. Training CatBoost...")
    catboost_model = CatBoostRegressor(
        iterations=100,
        depth=6,
        learning_rate=0.1,
        random_seed=42,
        verbose=False
    )
    catboost_model.fit(X_train, y_train)
    results['CatBoost'] = evaluate_model(catboost_model, X_test, y_test, 'CatBoost')
    results['CatBoost']['model'] = catboost_model
    
    return results

def plot_model_comparison(results):
    """
    Create comparison plots for model performance
    """
    models = list(results.keys())
    mae_scores = [results[model]['mae'] for model in models]
    rmse_scores = [results[model]['rmse'] for model in models]
    r2_scores = [results[model]['r2_score'] for model in models]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # MAE comparison
    axes[0].bar(models, mae_scores, color=['skyblue', 'lightgreen', 'lightcoral'])
    axes[0].set_title('Mean Absolute Error (MAE)')
    axes[0].set_ylabel('MAE')
    axes[0].tick_params(axis='x', rotation=45)
    
    # RMSE comparison
    axes[1].bar(models, rmse_scores, color=['skyblue', 'lightgreen', 'lightcoral'])
    axes[1].set_title('Root Mean Square Error (RMSE)')
    axes[1].set_ylabel('RMSE')
    axes[1].tick_params(axis='x', rotation=45)
    
    # R² comparison
    axes[2].bar(models, r2_scores, color=['skyblue', 'lightgreen', 'lightcoral'])
    axes[2].set_title('R² Score')
    axes[2].set_ylabel('R² Score')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Create results summary table
    summary_df = pd.DataFrame({
        'Model': models,
        'MAE': mae_scores,
        'RMSE': rmse_scores,
        'R² Score': r2_scores
    })
    
    return summary_df

print("Model training functions defined successfully!")

Model training functions defined successfully!


In [18]:
# Feature Importance Analysis
def analyze_feature_importance(results, feature_names, top_n=10):
    """
    Analyze and visualize feature importance from trained models
    """
    print("\n" + "="*50)
    print("FEATURE IMPORTANCE ANALYSIS")
    print("="*50)
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    for idx, (model_name, result) in enumerate(results.items()):
        model = result['model']
        
        if hasattr(model, 'feature_importances_'):
            # Get feature importances
            importances = model.feature_importances_
            
            # Create feature importance dataframe
            feature_importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False)
            
            # Plot top N features
            top_features = feature_importance_df.head(top_n)
            
            axes[idx].barh(range(len(top_features)), top_features['importance'], 
                          color=plt.cm.viridis(np.linspace(0, 1, len(top_features))))
            axes[idx].set_yticks(range(len(top_features)))
            axes[idx].set_yticklabels(top_features['feature'])
            axes[idx].set_xlabel('Feature Importance')
            axes[idx].set_title(f'{model_name} - Top {top_n} Features')
            axes[idx].invert_yaxis()
            
            print(f"\n{model_name} - Top {top_n} Important Features:")
            for i, (_, row) in enumerate(top_features.iterrows(), 1):
                print(f"{i:2d}. {row['feature']}: {row['importance']:.4f}")
    
    plt.tight_layout()
    plt.show()

def plot_predictions_vs_actual(results, y_test, dataset_name):
    """
    Plot predictions vs actual values for all models
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, (model_name, result) in enumerate(results.items()):
        y_pred = result['predictions']
        
        axes[idx].scatter(y_test, y_pred, alpha=0.6)
        axes[idx].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[idx].set_xlabel('Actual Values')
        axes[idx].set_ylabel('Predicted Values')
        axes[idx].set_title(f'{model_name} - Predictions vs Actual\n{dataset_name}')
        
        # Add R² score to the plot
        r2 = result['r2_score']
        axes[idx].text(0.05, 0.95, f'R² = {r2:.3f}', transform=axes[idx].transAxes, 
                      bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

def save_models_and_results(results, feature_names, dataset_name, models_dir='../models'):
    """
    Save trained models and results for later use
    """
    # Create models directory if it doesn't exist
    os.makedirs(models_dir, exist_ok=True)
    
    print(f"\nSaving models and results for {dataset_name}...")
    
    # Save each model
    for model_name, result in results.items():
        model = result['model']
        model_filename = f"{models_dir}/{dataset_name}_{model_name.replace(' ', '_').lower()}_model.joblib"
        joblib.dump(model, model_filename)
        print(f"  - Saved {model_name} model: {model_filename}")
    
    # Save feature names
    feature_filename = f"{models_dir}/{dataset_name}_feature_names.joblib"
    joblib.dump(feature_names, feature_filename)
    print(f"  - Saved feature names: {feature_filename}")
    
    # Save results summary
    summary_data = []
    for model_name, result in results.items():
        summary_data.append({
            'model_name': model_name,
            'mae': result['mae'],
            'rmse': result['rmse'],
            'r2_score': result['r2_score']
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_filename = f"{models_dir}/{dataset_name}_model_comparison.csv"
    summary_df.to_csv(summary_filename, index=False)
    print(f"  - Saved model comparison: {summary_filename}")
    
    return summary_df

print("Feature importance and model saving functions defined successfully!")

Feature importance and model saving functions defined successfully!


In [19]:
# LAND DATASET MODELING
print("="*60)
print("LAND DATASET MODELING")
print("="*60)

# First, let's identify the target column for land dataset
print("Land dataset columns:")
for i, col in enumerate(df_land_processed.columns, 1):
    print(f"{i:2d}. {col}")

# We need to identify the price/target column
# Look for columns that might represent price
price_related_cols = [col for col in df_land_processed.columns 
                     if any(keyword in col.lower() for keyword in ['harga', 'price', 'nilai', 'value', 'njop', 'rp'])]

print(f"\nPotential price columns in land dataset: {price_related_cols}")

# Check for any actual non-null values in NJOP column
if 'NJOP_Rp_per_m2' in df_land_processed.columns:
    njop_stats = df_land_processed['NJOP_Rp_per_m2'].describe()
    print(f"\nNJOP_Rp_per_m2 statistics:")
    print(njop_stats)
    print(f"Non-null count: {df_land_processed['NJOP_Rp_per_m2'].count()}")
    print(f"Total rows: {len(df_land_processed)}")

# Let's check all numeric columns for actual data
print(f"\nChecking numeric columns for non-null values:")
for col in numeric_cols_land:
    non_null_count = df_land_processed[col].count()
    print(f"{col}: {non_null_count}/{len(df_land_processed)} non-null values")

# Show a sample of the original (unprocessed) data to understand the structure
print(f"\nOriginal land dataset sample:")
print(df_land.head())

# For demonstration, let's assume the first numeric column is our target
# In practice, you should examine the data and select the appropriate target
numeric_cols_land = df_land_processed.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric columns in land dataset: {numeric_cols_land}")

# Let's examine the first few rows to better understand the data structure
print("\nFirst few rows of processed land dataset:")
print(df_land_processed.head())

# We'll need to manually identify the target column based on the actual data structure
# For now, let's create a placeholder to demonstrate the workflow
print("\nNote: Target column needs to be identified based on actual data structure.")
print("Please examine the data above and update the target_column_land variable below.")

LAND DATASET MODELING
Land dataset columns:
 1. Kecamatan
 2. Lokasi
 3. Luas_m2
 4. Sertifikat
 5. Kondisi
 6. Tipe_Iklan
 7. NJOP_Rp_per_m2
 8. Jumlah_Penduduk
 9. Aksesibilitas

Potential price columns in land dataset: ['NJOP_Rp_per_m2']

NJOP_Rp_per_m2 statistics:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: NJOP_Rp_per_m2, dtype: float64
Non-null count: 0
Total rows: 7999

Checking numeric columns for non-null values:
Lokasi: 0/7999 non-null values
Luas_m2: 0/7999 non-null values
Sertifikat: 0/7999 non-null values
Kondisi: 0/7999 non-null values
Tipe_Iklan: 0/7999 non-null values
NJOP_Rp_per_m2: 0/7999 non-null values
Jumlah_Penduduk: 0/7999 non-null values
Aksesibilitas: 0/7999 non-null values

Original land dataset sample:
                                           Kecamatan  Lokasi  Luas_m2  \
0  Gubeng,"Jl. Tubagus Ismail, Gubeng, Surabaya",...     NaN      NaN   
1  Sawahan,"Jl. Siliwangi, Sawahan, Surabaya",112

In [20]:
# Execute Land Dataset Modeling
# Update this variable with the correct target column name after examining the data
target_column_land = 'NJOP_Rp_per_m2'  # NJOP (Nilai Jual Objek Pajak) per square meter

# Check if we have a valid target column
if target_column_land and target_column_land in df_land_processed.columns:
    print(f"\nUsing '{target_column_land}' as target variable for land dataset")
    
    # Prepare features and target
    X_land, y_land, feature_names_land = prepare_features_target(
        df_land_processed, 
        target_column_land,
        exclude_columns=['Kecamatan', 'Lokasi', 'Sertifikat', 'Kondisi', 'Tipe_Iklan', 'Aksesibilitas']  # Exclude categorical columns
    )
    
    # Split the data
    X_train_land, X_test_land, y_train_land, y_test_land = train_test_split(
        X_land, y_land, test_size=0.2, random_state=42
    )
    
    print(f"\nTraining set size: {X_train_land.shape[0]}")
    print(f"Test set size: {X_test_land.shape[0]}")
    
    # Scale features
    scaler_land = StandardScaler()
    X_train_land_scaled = scaler_land.fit_transform(X_train_land)
    X_test_land_scaled = scaler_land.transform(X_test_land)
    
    # Convert back to DataFrame to maintain feature names
    X_train_land_scaled = pd.DataFrame(X_train_land_scaled, columns=feature_names_land)
    X_test_land_scaled = pd.DataFrame(X_test_land_scaled, columns=feature_names_land)
    
    # Train models
    land_results = train_models(X_train_land_scaled, X_test_land_scaled, y_train_land, y_test_land)
    
    # Visualize results
    land_summary = plot_model_comparison(land_results)
    print("\nLand Dataset Model Comparison:")
    print(land_summary)
    
    # Feature importance analysis
    analyze_feature_importance(land_results, feature_names_land)
    
    # Plot predictions vs actual
    plot_predictions_vs_actual(land_results, y_test_land, "Land Dataset")
    
    # Perform EDA on target variable
    create_eda_plots(df_land_processed, "LAND DATASET", target_column_land)
    
    # Save models and results
    land_summary_saved = save_models_and_results(land_results, feature_names_land, "land")
    
else:
    print("\nPlease update the 'target_column_land' variable with the correct target column name.")
    print("Available columns:", df_land_processed.columns.tolist())


Using 'NJOP_Rp_per_m2' as target variable for land dataset
Features selected: ['Luas_m2', 'Jumlah_Penduduk']
Feature matrix shape: (7999, 2)
Target shape: (7999,)

Training set size: 6399
Test set size: 1600
Training machine learning models...

1. Training Random Forest...


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# BUILDING DATASET MODELING
print("\n" + "="*60)
print("BUILDING DATASET MODELING")
print("="*60)

# Identify the target column for building dataset
print("Building dataset columns:")
for i, col in enumerate(df_building_processed.columns, 1):
    print(f"{i:2d}. {col}")

# Look for potential price columns
price_related_cols_building = [col for col in df_building_processed.columns 
                              if any(keyword in col.lower() for keyword in ['harga', 'price', 'nilai', 'value'])]

print(f"\nPotential price columns in building dataset: {price_related_cols_building}")

# Numeric columns
numeric_cols_building = df_building_processed.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric columns in building dataset: {numeric_cols_building}")

print("\nFirst few rows of processed building dataset:")
print(df_building_processed.head())

In [ ]:
# Execute Building Dataset Modeling
# Update this variable with the correct target column name after examining the data
target_column_building = 'NJOP_Rp_per_m2'  # NJOP (Nilai Jual Objek Pajak) per square meter

# Check if we have a valid target column
if target_column_building and target_column_building in df_building_processed.columns:
    print(f"\nUsing '{target_column_building}' as target variable for building dataset")
    
    # Prepare features and target
    X_building, y_building, feature_names_building = prepare_features_target(
        df_building_processed, 
        target_column_building,
        exclude_columns=['Kecamatan', 'Alamat', 'Sertifikat', 'Kondisi Perabotan', 'Tipe Iklan', 'Aksesibilitas', 'Hadap', 'Hook', 'Kondisi Properti', 'Sumber Air']  # Exclude categorical columns
    )
    
    # Split the data
    X_train_building, X_test_building, y_train_building, y_test_building = train_test_split(
        X_building, y_building, test_size=0.2, random_state=42
    )
    
    print(f"\nTraining set size: {X_train_building.shape[0]}")
    print(f"Test set size: {X_test_building.shape[0]}")
    
    # Scale features
    scaler_building = StandardScaler()
    X_train_building_scaled = scaler_building.fit_transform(X_train_building)
    X_test_building_scaled = scaler_building.transform(X_test_building)
    
    # Convert back to DataFrame to maintain feature names
    X_train_building_scaled = pd.DataFrame(X_train_building_scaled, columns=feature_names_building)
    X_test_building_scaled = pd.DataFrame(X_test_building_scaled, columns=feature_names_building)
    
    # Train models
    building_results = train_models(X_train_building_scaled, X_test_building_scaled, y_train_building, y_test_building)
    
    # Visualize results
    building_summary = plot_model_comparison(building_results)
    print("\nBuilding Dataset Model Comparison:")
    print(building_summary)
    
    # Feature importance analysis
    analyze_feature_importance(building_results, feature_names_building)
    
    # Plot predictions vs actual
    plot_predictions_vs_actual(building_results, y_test_building, "Building Dataset")
    
    # Perform EDA on target variable
    create_eda_plots(df_building_processed, "BUILDING DATASET", target_column_building)
    
    # Save models and results
    building_summary_saved = save_models_and_results(building_results, feature_names_building, "building")
    
else:
    print("\nPlease update the 'target_column_building' variable with the correct target column name.")
    print("Available columns:", df_building_processed.columns.tolist())

In [ ]:
# RESULTS SUMMARY AND DASHBOARD INTEGRATION
print("\n" + "="*60)
print("RESULTS SUMMARY AND DASHBOARD INTEGRATION")
print("="*60)

# Create comprehensive summary
def create_comprehensive_summary():
    """
    Create a comprehensive summary of all modeling results
    """
    summary_data = []
    
    # Add land results if available
    if 'land_results' in globals() and land_results:
        for model_name, result in land_results.items():
            summary_data.append({
                'Dataset': 'Land',
                'Model': model_name,
                'MAE': result['mae'],
                'RMSE': result['rmse'],
                'R² Score': result['r2_score']
            })
    
    # Add building results if available
    if 'building_results' in globals() and building_results:
        for model_name, result in building_results.items():
            summary_data.append({
                'Dataset': 'Building',
                'Model': model_name,
                'MAE': result['mae'],
                'RMSE': result['rmse'],
                'R² Score': result['r2_score']
            })
    
    if summary_data:
        overall_summary = pd.DataFrame(summary_data)
        
        print("Overall Model Performance Summary:")
        print(overall_summary.to_string(index=False))
        
        # Find best models
        if len(overall_summary) > 0:
            best_r2_model = overall_summary.loc[overall_summary['R² Score'].idxmax()]
            best_mae_model = overall_summary.loc[overall_summary['MAE'].idxmin()]
            
            print(f"\nBest Model by R² Score:")
            print(f"  {best_r2_model['Dataset']} - {best_r2_model['Model']} (R² = {best_r2_model['R² Score']:.4f})")
            
            print(f"\nBest Model by MAE:")
            print(f"  {best_mae_model['Dataset']} - {best_mae_model['Model']} (MAE = {best_mae_model['MAE']:,.2f})")
        
        return overall_summary
    else:
        print("No model results available. Please run the modeling sections first.")
        return None

# Export prediction function for dashboard integration
def create_prediction_function():
    """
    Create prediction functions for dashboard integration
    """
    prediction_code = '''
# Prediction functions for dashboard integration
import joblib
import pandas as pd
import numpy as np

def load_model_and_predict(model_type, dataset_type, input_data):
    """
    Load trained model and make predictions
    
    Args:
        model_type: 'random_forest', 'xgboost', or 'catboost'
        dataset_type: 'land' or 'building'
        input_data: Dictionary or DataFrame with feature values
    
    Returns:
        prediction: Predicted price value
    """
    try:
        # Load the model
        model_path = f"../models/{dataset_type}_{model_type}_model.joblib"
        model = joblib.load(model_path)
        
        # Load feature names
        features_path = f"../models/{dataset_type}_feature_names.joblib"
        feature_names = joblib.load(features_path)
        
        # Prepare input data
        if isinstance(input_data, dict):
            input_df = pd.DataFrame([input_data])
        else:
            input_df = input_data.copy()
        
        # Ensure all required features are present
        for feature in feature_names:
            if feature not in input_df.columns:
                input_df[feature] = 0  # Default value for missing features
        
        # Select only the required features in the correct order
        input_df = input_df[feature_names]
        
        # Make prediction
        prediction = model.predict(input_df)
        
        return prediction[0] if len(prediction) == 1 else prediction
        
    except Exception as e:
        print(f"Error making prediction: {e}")
        return None

def predict_land_price(luas_tanah, lokasi_encoded=0, **kwargs):
    """
    Predict land price based on area and other features
    """
    input_data = {
        'luas_tanah': luas_tanah,
        'lokasi_encoded': lokasi_encoded,
        **kwargs
    }
    
    # Try different models and return the best result
    models = ['random_forest', 'xgboost', 'catboost']
    predictions = {}
    
    for model in models:
        pred = load_model_and_predict(model, 'land', input_data)
        if pred is not None:
            predictions[model] = pred
    
    return predictions

def predict_building_price(luas_bangunan, luas_tanah, lokasi_encoded=0, **kwargs):
    """
    Predict building price based on building area, land area, and other features
    """
    input_data = {
        'luas_bangunan': luas_bangunan,
        'luas_tanah': luas_tanah,
        'lokasi_encoded': lokasi_encoded,
        'building_to_land_ratio': luas_bangunan / (luas_tanah + 1),
        **kwargs
    }
    
    # Try different models and return the best result
    models = ['random_forest', 'xgboost', 'catboost']
    predictions = {}
    
    for model in models:
        pred = load_model_and_predict(model, 'building', input_data)
        if pred is not None:
            predictions[model] = pred
    
    return predictions
'''
    
    # Save prediction functions to a Python file
    with open('../app/prediction_functions.py', 'w') as f:
        f.write(prediction_code)
    
    print("Prediction functions saved to '../app/prediction_functions.py'")
    print("These functions can be imported into your Flask application for dashboard integration.")

# Execute summary and integration preparation
overall_summary = create_comprehensive_summary()
create_prediction_function()

print("\n" + "="*60)
print("MODELING WORKFLOW COMPLETED!")
print("="*60)
print("\nNext Steps for Dashboard Integration:")
print("1. Update target column variables with actual column names from your datasets")
print("2. Run the modeling sections after identifying target columns")
print("3. Import prediction functions into your Flask app:")
print("   from app.prediction_functions import predict_land_price, predict_building_price")
print("4. Use the saved models for real-time predictions in your admin dashboard")
print("\nModel files will be saved in the '../models/' directory")
print("Prediction functions are available in '../app/prediction_functions.py'")

In [ ]:
# Final Data Examination and Model Setup Instructions
print("="*80)
print("PROPERTY PRICE PREDICTION MODEL - SETUP COMPLETE")
print("="*80)

print("\n📊 DATASET OVERVIEW:")
print(f"Land Dataset: {df_land.shape[0]} rows, {df_land.shape[1]} columns")
print(f"Building Dataset: {df_building.shape[0]} rows, {df_building.shape[1]} columns")

print("\n🏗️ NOTEBOOK STRUCTURE:")
print("✅ 1. Library imports and setup")
print("✅ 2. Data loading and exploration")
print("✅ 3. Data quality assessment")
print("✅ 4. Exploratory data analysis functions")
print("✅ 5. Data preprocessing pipeline")
print("✅ 6. Model training functions (XGBoost, Random Forest, CatBoost)")
print("✅ 7. Feature importance analysis")
print("✅ 8. Model evaluation and comparison")
print("✅ 9. Results export for dashboard integration")

print("\n🎯 TARGET VARIABLES IDENTIFIED:")
print("Land Dataset: NJOP_Rp_per_m2 (Tax Object Sale Value per m²)")
print("Building Dataset: NJOP_Rp_per_m2 (Tax Object Sale Value per m²)")

print("\n📋 NEXT STEPS:")
print("1. Examine the actual data values in both datasets")
print("2. If NJOP_Rp_per_m2 has valid data, proceed with modeling")
print("3. If NJOP_Rp_per_m2 is not suitable, identify alternative target columns")
print("4. Run the modeling sections after confirming target variables")
print("5. Use the trained models for dashboard integration")

print("\n🔧 FEATURES AVAILABLE:")
print("\nLand Dataset Features:")
land_features = [col for col in df_land.columns if col != 'NJOP_Rp_per_m2']
for i, feature in enumerate(land_features, 1):
    print(f"  {i}. {feature}")

print("\nBuilding Dataset Features:")
building_features = [col for col in df_building.columns if col != 'NJOP_Rp_per_m2']
for i, feature in enumerate(building_features, 1):
    print(f"  {i}. {feature}")

print("\n🚀 READY FOR MODELING!")
print("The notebook is now complete with all necessary functions.")
print("Modify the target column variables as needed and run the modeling sections.")

# Show sample data to help identify the best approach
print("\n📋 SAMPLE DATA PREVIEW:")
print("\nLand Dataset Sample:")
print(df_land.head(3))
print("\nBuilding Dataset Sample:")
print(df_building.head(3))

PROPERTY PRICE PREDICTION MODEL - SETUP COMPLETE

📊 DATASET OVERVIEW:
Land Dataset: 8000 rows, 9 columns
Building Dataset: 8980 rows, 22 columns

🏗️ NOTEBOOK STRUCTURE:
✅ 1. Library imports and setup
✅ 2. Data loading and exploration
✅ 3. Data quality assessment
✅ 4. Exploratory data analysis functions
✅ 5. Data preprocessing pipeline
✅ 6. Model training functions (XGBoost, Random Forest, CatBoost)
✅ 7. Feature importance analysis
✅ 8. Model evaluation and comparison
✅ 9. Results export for dashboard integration

🎯 TARGET VARIABLES IDENTIFIED:
Land Dataset: NJOP_Rp_per_m2 (Tax Object Sale Value per m²)
Building Dataset: NJOP_Rp_per_m2 (Tax Object Sale Value per m²)

📋 NEXT STEPS:
1. Examine the actual data values in both datasets
2. If NJOP_Rp_per_m2 has valid data, proceed with modeling
3. If NJOP_Rp_per_m2 is not suitable, identify alternative target columns
4. Run the modeling sections after confirming target variables
5. Use the trained models for dashboard integration

🔧 FEATURES 